#### Convert "mean_values" excel page to .csv file then run this script with location of input filed mapped at bottom

In [2]:
import pandas as pd
import re
import os

def process_csv_for_prism(input_csv_path):
    """
    Reads a CSV, reshapes the data by stripping numerical suffixes from sample names,
    and saves each metric as a separate sheet in an Excel file ready for GraphPad Prism.
    Output file is automatically named <InputFileName>_Prism.xlsx in the same folder.
    """
    
    # --- 1. Dynamic Output Path Generation ---
    # Split the input path into (root + filename_without_extension) and (extension)
    # e.g., "C:\...\Data.csv" -> "C:\...\Data"
    base_path = os.path.splitext(input_csv_path)[0]
    
    # Add the requested suffix and correct extension
    output_excel_path = f"{base_path}_Prism.xlsx"

    # --- 2. Load the dataset ---
    try:
        df = pd.read_csv(input_csv_path)
        print(f"Loaded data from: {input_csv_path}")
    except FileNotFoundError:
        print(f"Error: The file was not found at {input_csv_path}")
        return

    # --- 3. Define the exact column order requested ---
    desired_order = [
        "P2A PreISO", "P2A PostISO", 
        "SUPH222P PreISO", "SUPH222P PostISO", 
        "CRISPH222P PreISO", "CRISPH222P PostISO", 
        "SUPR541C PreISO", "SUPR541C PostISO"
    ]

    # --- 4. Identify metric columns ---
    # Exclude 'ROI', 'Sample', and any column ending in '_std'
    metric_columns = [
        col for col in df.columns 
        if "ROI" not in col and not col.endswith("_std") and col != "Sample"
    ]

    # Initialize the Excel Writer
    with pd.ExcelWriter(output_excel_path, engine='openpyxl') as writer:
        
        # Iterate through each metric (e.g., 'APD90_s_mean')
        for metric in metric_columns:
            
            # Dictionary to hold data lists for this sheet
            data_map = {key: [] for key in desired_order}
            
            # Iterate through rows in the original CSV
            for index, row in df.iterrows():
                # Get the raw sample name
                raw_name = str(row['Sample'])
                
                # --- CLEANING STEPS ---
                # 1. Remove trailing numbers (e.g., 001, 002)
                clean_name = re.sub(r'\d+$', '', raw_name)
                
                # 2. Fix specific naming inconsistency (Pre_ISO -> PreISO)
                clean_name = clean_name.replace('Pre_ISO', 'PreISO')
                
                # 3. Replace underscores with spaces to match headers
                clean_name = clean_name.replace('_', ' ').strip()

                # Add data to the correct list if the name matches our columns
                if clean_name in data_map:
                    data_map[clean_name].append(row[metric])
            
            # Convert to DataFrame
            # We use a dictionary of Series to handle columns of unequal lengths
            sheet_df = pd.DataFrame({k: pd.Series(v) for k, v in data_map.items()})

            # Ensure strict column ordering
            sheet_df = sheet_df[desired_order]

            # Write to Excel Sheet (Sheet name limit is 31 chars)
            sheet_name = metric[:31]
            sheet_df.to_excel(writer, sheet_name=sheet_name, index=False)

    print(f"Processing complete.")
    print(f"Output saved to: {output_excel_path}")

# --- Main Execution ---
if __name__ == "__main__":
    # Your specific file path
    input_file = r"C:\Users\m254292\OneDrive - Mayo Clinic\Python\Scripts_APD_Calcium\Scripts_APD_Calcium\Nikon\ToPrism\7102025_Sup Rep BeRST_ISO500_Mean Values.csv"
    
    # Run the function (Output path is now handled automatically inside)
    process_csv_for_prism(input_file)

Loaded data from: C:\Users\m254292\OneDrive - Mayo Clinic\Python\Scripts_APD_Calcium\Scripts_APD_Calcium\Nikon\ToPrism\7102025_Sup Rep BeRST_ISO500_Mean Values.csv
Processing complete.
Output saved to: C:\Users\m254292\OneDrive - Mayo Clinic\Python\Scripts_APD_Calcium\Scripts_APD_Calcium\Nikon\ToPrism\7102025_Sup Rep BeRST_ISO500_Mean Values_Prism.xlsx
